In [ ]:
# ======================================================================
# 🧠 Logic Field Theory (LFT) – Hardware Validation Notebook (June 2025)
# ======================================================================
# Fully compatible with latest IBM Quantum Platform (SamplerV2 + ISA)
# ----------------------------------------------------------------------
# • Requires API key and instance CRN from https://quantum.ibm.com/account
# • Uses SamplerV2 primitive (V1 deprecated)
# • Requires ISA transpilation (mandatory since March 2024)
# • Saves results to Google Drive under /content/drive/MyDrive/LFT-Gen14
# ======================================================================

# 🧰 Setup: Install latest packages
!pip install qiskit qiskit-ibm-runtime matplotlib seaborn pandas -U -q

# 📁 Mount Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# 📦 Imports
import os, time, json, warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2
from qiskit.visualization import plot_histogram
warnings.filterwarnings('ignore')

# 📂 Setup project folders
project_folder = "/content/drive/MyDrive/LFT-Gen14"
subfolders = ['data', 'results', 'plots', 'hardware_logs']
folder_paths = { name: os.path.join(project_folder, name) for name in subfolders }
for path in folder_paths.values():
    os.makedirs(path, exist_ok=True)

# 🔐 IBM Quantum Credentials (Modern Platform - June 2025)
IBM_API_KEY = "PASTE_YOUR_API_KEY_HERE"
IBM_INSTANCE = "PASTE_YOUR_INSTANCE_CRN_HERE"

if ("PASTE" in IBM_API_KEY or "PASTE" in IBM_INSTANCE):
    raise ValueError("❌ ERROR: Please replace IBM_API_KEY and IBM_INSTANCE with your actual credentials.")

# 🛰️ Connect to IBM Quantum Platform
try:
    service = QiskitRuntimeService(
        channel="ibm_quantum_platform",
        token=IBM_API_KEY,
        instance=IBM_INSTANCE
    )
    print("✅ Connected to IBM Quantum Platform")
except Exception as e:
    raise RuntimeError(f"❌ Connection failed: {e}")

# 🔍 Select optimal backend
def select_best_backend(service, min_qubits=2, max_queue=100):
    candidates = []
    for backend in service.backends(operational=True, simulator=False):
        try:
            status = backend.status()
            if backend.num_qubits >= min_qubits and status.pending_jobs <= max_queue:
                candidates.append((backend, status.pending_jobs))
        except: continue
    if not candidates:
        raise RuntimeError("❌ No suitable hardware backend found.")
    selected = sorted(candidates, key=lambda x: (x[1], -x[0].num_qubits))[0][0]
    print(f"🎯 Selected backend: {selected.name} ({selected.num_qubits} qubits)")
    return selected

quantum_backend = select_best_backend(service)

# ✅ Test SamplerV2 instantiation
try:
    sampler_test = SamplerV2(backend=quantum_backend)
    print("✅ SamplerV2 is supported on this backend")
except Exception as e:
    raise RuntimeError(f"❌ SamplerV2 setup failed: {e}")

# 🧪 Define a simple Bell state test circuit
qc = QuantumCircuit(2, 2)
qc.h(0)
qc.cx(0, 1)
qc.measure([0,1], [0,1])

# 🔧 Transpile using ISA-compliant pass manager
pm = generate_preset_pass_manager(backend=quantum_backend, optimization_level=1)
isa_qc = pm.run(qc)

# 🚀 Run job using SamplerV2
sampler = SamplerV2(backend=quantum_backend)
job = sampler.run([isa_qc], shots=1024)
print(f"📡 Job submitted: {job.job_id()}")
result = job.result()

# 📊 Parse result
counts = result[0].data.meas.get_counts() if hasattr(result[0].data, "meas") else {k: int(v * 1024) for k, v in result[0].quasi_dists[0].items()}
print("🧾 Counts:", counts)

# 💾 Save results to Drive
now = datetime.now().strftime("%Y%m%d_%H%M%S")
path = os.path.join(folder_paths['results'], f'bell_test_result_{now}.json')
with open(path, 'w') as f:
    json.dump(counts, f)
print(f"✅ Results saved to: {path}")

# 📈 Plot histogram
plot_histogram(counts)
